In [51]:
import numpy as np
import pandas as pd
import tushare as ts

from tqdm import tqdm
import warnings
import time

import codecs
import ast
import jieba
import datetime
from gensim.models import word2vec
from gensim.models import FastText
from glove import Glove, Corpus

pro = ts.pro_api('52119551ebc32c734a42fd61ccf33344fde3c6195b0aacec58d15140')
STARTDATE = '20190101'
ENDDATE = '20190622'

warnings.filterwarnings("ignore")
stopwords = [line.strip() for line in codecs.open(
    r'./stopwords.txt', 'r', 'utf-8').readlines()]

# Import previous data

In [84]:
start = datetime.datetime(2014, 4, 1, 0, 0, 0)
end = datetime.datetime(2019, 6, 22, 0, 0, 0)

delta = end - start

def get_new(news):
    news_text = []
    for i in tqdm(range(0, delta.days + 1)):
        startdate = (start + datetime.timedelta(days=i)).date()
        enddate = (start + datetime.timedelta(days=i+1)).date()
        df = pro.news(src=news, start_date=str(startdate), end_date=str(enddate))
        news_text.append(df)
        time.sleep(0.5)
    return news_text

In [85]:
df_10jqka_list = get_new('10jqka')

100%|██████████| 1909/1909 [34:36<00:00,  1.14s/it] 


In [86]:
df_eastmoney_list = get_new('eastmoney')

100%|██████████| 1909/1909 [36:37<00:00,  1.39s/it] 


In [88]:
df_sina_list = get_new('sina')


  8%|▊         | 150/1909 [04:55<48:56,  1.67s/it]


 16%|█▌        | 300/1909 [08:52<46:16,  1.73s/it]


 24%|██▎       | 451/1909 [11:57<24:43,  1.02s/it]


 32%|███▏      | 602/1909 [14:42<24:36,  1.13s/it]


 39%|███▉      | 753/1909 [17:32<22:58,  1.19s/it]


...

 71%|███████   | 1350/1909 [28:37<10:46,  1.16s/it]


 78%|███████▊  | 1498/1909 [31:50<11:59,  1.75s/it]


 86%|████████▌ | 1646/1909 [35:34<12:52,  2.94s/it]


 94%|█████████▍| 1794/1909 [40:44<03:28,  1.82s/it]


100%|██████████| 1909/1909 [45:05<00:00,  1.52s/it]


In [89]:
df_sina = pd.concat(df_sina_list)
df_10jqka = pd.concat(df_10jqka_list)
df_eastmoney = pd.concat(df_eastmoney_list)

In [90]:
df_sina.to_csv(r'./sina_news.csv')
df_10jqka.to_csv(r'./10jqka_news.csv')
df_eastmoney.to_csv(r'./eastmoney_news.csv')

In [91]:
# text_df = [df_sina, df_10jqka, df_eastmoney, df_yuncaijing]
text_df = [df_sina, df_10jqka, df_eastmoney]

In [92]:
for i, df in enumerate(text_df):
    df['datetime'] = df['datetime'].str.split(' ').apply(lambda x: x[0])
    df[str(i)+'_content'] = df['content']
    df[str(i)+'_title'] = df['title']
    df.drop(columns=["content", "title"], inplace=True)

In [93]:
df_sina.columns.values, df_10jqka.columns.values, df_eastmoney.columns.values,

(array(['datetime', '0_content', '0_title'], dtype=object),
 array(['datetime', '1_content', '1_title'], dtype=object),
 array(['datetime', '2_content', '2_title'], dtype=object))

In [94]:
merge_date_sina = df_sina[['datetime', '0_content']]
merge_date_sina = merge_date_sina.groupby(
    ['datetime'])['0_content'].apply(';'.join).reset_index()

merge_date_sina_title = df_sina[['datetime', '0_title']]
merge_date_sina_title = merge_date_sina_title.groupby(
    ['datetime'])['0_title'].apply(';'.join).reset_index()
merge_date_sina = merge_date_sina.merge(merge_date_sina_title, on="datetime", how="left")
merge_date_sina['0_text'] = merge_date_sina[['0_content', '0_title']].apply(lambda x: ';'.join(x), axis=1)
merge_date_sina.drop(columns=["0_content", "0_title"], inplace=True)


merge_date_10jqka = df_10jqka[['datetime', '1_content']]
merge_date_10jqka['1_content'] = merge_date_10jqka['1_content'].astype(str)
merge_date_10jqka = merge_date_10jqka.groupby(
    ['datetime'])['1_content'].apply(';'.join).reset_index()

merge_date_10jqka_title = df_10jqka[['datetime', '1_title']]
merge_date_10jqka_title = merge_date_10jqka_title.groupby(
    ['datetime'])['1_title'].apply(';'.join).reset_index()
merge_date_10jqka = merge_date_10jqka.merge(merge_date_10jqka_title, on="datetime", how="left")
merge_date_10jqka['1_text'] = merge_date_10jqka[['1_content', '1_title']].apply(lambda x: ';'.join(x), axis=1)
merge_date_10jqka.drop(columns=["1_content", "1_title"], inplace=True)


merge_date_eastmoney = df_eastmoney[['datetime', '2_content']]
merge_date_eastmoney = merge_date_eastmoney.groupby(
    ['datetime'])['2_content'].apply(';'.join).reset_index()

merge_date_eastmoney_title = df_eastmoney[['datetime', '2_title']]
merge_date_eastmoney_title = merge_date_eastmoney_title.groupby(
    ['datetime'])['2_title'].apply(';'.join).reset_index()
merge_date_eastmoney = merge_date_eastmoney.merge(merge_date_eastmoney_title, on="datetime", how="left")
merge_date_eastmoney['2_text'] = merge_date_eastmoney[['2_content', '2_title']].apply(lambda x: ';'.join(x), axis=1)
merge_date_eastmoney.drop(columns=["2_content", "2_title"], inplace=True)

In [95]:
merge_date_sina = merge_date_sina.merge(merge_date_10jqka, on="datetime", how="left")
merge_date_sina = merge_date_sina.merge(merge_date_eastmoney, on="datetime", how="left")
merge_date_sina.head()

,datetime,0_text,1_text,2_text
0,2018-10-08,恐慌指数VIX报18.38，触及6月28日以来日内最高点。;道指跌超200点。;道指跌150...,NaN,NaN
1,2018-10-09,在岸人民币兑美元（CNY）北京时间23:30收报6.9225元，较周一夜盘收盘涨60点；全天...,NaN,NaN
2,2018-10-10,JCP上涨13.8%。;Loop Industries（LOOP）上涨18.4%，该公司与百...,NaN,NaN
3,2018-10-11,现货白银扩大涨幅至2%，现报14.56美元/盎司。;纳指再度转涨。;哥伦比亚广播下跌2....,NaN,NaN
4,2018-10-12,花旗：信贷资本应当会在四季度保持稳定。;美元兑日元下跌逾0.1%，刷新日低至112，较日高回...,NaN,NaN


In [96]:
merge_date_sina = merge_date_sina.fillna('')
merge_date_sina['text'] = merge_date_sina[['0_text', '1_text', '2_text']].apply(lambda x: ';'.join(x), axis=1)
merge_date_sina.drop(columns=['0_text', '1_text', '2_text'], inplace=True)
merge_date_sina.head()

,datetime,text
0,2018-10-08,恐慌指数VIX报18.38，触及6月28日以来日内最高点。;道指跌超200点。;道指跌150...
1,2018-10-09,在岸人民币兑美元（CNY）北京时间23:30收报6.9225元，较周一夜盘收盘涨60点；全天...
2,2018-10-10,JCP上涨13.8%。;Loop Industries（LOOP）上涨18.4%，该公司与百...
3,2018-10-11,现货白银扩大涨幅至2%，现报14.56美元/盎司。;纳指再度转涨。;哥伦比亚广播下跌2....
4,2018-10-12,花旗：信贷资本应当会在四季度保持稳定。;美元兑日元下跌逾0.1%，刷新日低至112，较日高回...


In [97]:
text_df = merge_date_sina.copy()

In [98]:
text_df.shape

(258, 2)

In [99]:
text_df.to_csv('./media_news.csv', index=False)

# Segment

In [56]:
text_df = pd.read_csv(r'./media_news.csv')
text_df.head()

,datetime,text
0,2018-10-09,在岸人民币兑美元（CNY）北京时间23:30收报6.9225元，较周一夜盘收盘涨60点；全天...
1,2018-10-11,现货白银扩大涨幅至2%，现报14.56美元/盎司。;纳指再度转涨。;哥伦比亚广播下跌2....
2,2018-10-13,意大利副总理Salvini：预算法律可能会在周末之前准备就绪。;【粤港澳大湾区现代化交通运输...
3,2018-10-15,摩根大通策略师：波动率上升释放买入标普500指数的信号。;美国纽约联储GDPNowcast模...
4,2018-10-17,道指跌幅收窄至不到60点。;特斯拉跌3.2%。;美国财政部雇员因为泄露敏感信息而被捕。;立陶...


In [100]:
def seg_func(x):
    return ' '.join(list(filter(lambda w: w not in stopwords and w != ' ', jieba.cut(x, cut_all=False))))

In [101]:
text_df['text'] = (text_df['text'].astype(str)).apply(seg_func)
text_df.head()

,datetime,text
0,2018-10-08,恐慌 指数 VIX 报 18.38 触及 6 月 28 日 以来 日内 最高点 道 指跌超 ...
1,2018-10-09,在岸 人民币 兑 美元 CNY 北京 时间 23 30 收报 6.9225 元 较周 一夜 ...
2,2018-10-10,JCP 上涨 13.8% Loop Industries LOOP 上涨 18.4% 公司 ...
3,2018-10-11,现货 白银 扩大 涨幅 2% 现报 14.56 美元 盎司 纳指 再度 转涨 哥伦比亚 广播...
4,2018-10-12,花旗 信贷 资本 应当 会 四季度 保持稳定 美元兑 日元 下跌 逾 0.1% 刷新 日低 ...


In [106]:
text_df.to_csv('./media_news_seg.csv', index=False)

# Train TF-IDF

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [108]:
tfidf_v = TfidfVectorizer()
tfidf_text_df = tfidf_v.fit_transform(text_df['text'])

In [109]:
tfidf_text_df.shape

(258, 142892)

## Save tfidf model

In [110]:
pickle.dump(tfidf_text_df, open("media_tfidf.pkl", "wb"))

## Load tfidf model and update model

In [ ]:
tf1 = pickle.load(open("media_tfidf.pkl", 'rb'))

# Create new tfidfVectorizer with old vocabulary
tf1_new = TfidfVectorizer(vocabulary = tf1.vocabulary_)
X_tf1 = tf1_new.fit_transform(new_corpus)

## Get keyword of each day

In [68]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [80]:
# you only needs to do this once, this is a mapping of index to 
feature_names = tfidf_v.get_feature_names()

In [81]:
def get_keywords(doc, topn=50):
    # #generate tf-idf for the given document
    tf_idf_vector_doc = tfidf_v.transform([doc])

    # #sort the tf-idf vectors by descending order of scores
    sorted_items = sort_coo(tf_idf_vector_doc.tocoo())

    # #extract only the top n; n here is 10
    keywords = extract_topn_from_vector(feature_names, sorted_items, topn)
    return keywords

# PCA decomposition

# Prepare target